In [ ]:
!pip install tfx

In [ ]:
!unzip modules.zip

Archive:  modules.zip
replace modules/components_tfx.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os
from typing import Text
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from modules import components_tfx

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving heart.csv to heart.csv


In [ ]:
import pandas as pd

real_df = pd.read_csv("/content/data/heart.csv")

real_df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [ ]:
import pandas as pd

df = pd.read_csv("/content/data/heart.csv")

print(df.isnull().sum())


Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64


In [ ]:
df = df.dropna()
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [ ]:
DATA_PATH = 'data'

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

df.to_csv(os.path.join(DATA_PATH, '/content/data/heart.csv'), index=False)
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [ ]:
PIPELINE_NAME = "heart-failure-pipeline"

# Pipeline inputs
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

# Pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

In [ ]:
def init_local_pipeline(
    components, pipeline_root: Text
) -> pipeline.Pipeline:
    """Init local pipeline

    Args:
        components (dict): tfx components
        pipeline_root (Text): path to pipeline directory

    Returns:
        pipeline.Pipeline: apache beam pipeline orchestration
    """
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        '--direct_running_mode=multi_processing'
        # 0 auto-detect based on on the number of CPUs available
        # during execution time.
        '----direct_num_workers=0'
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        )
    )

In [ ]:
logging.set_verbosity(logging.INFO)

components = components_tfx.init_components({
    "data_dir": DATA_ROOT,
    "trainer_module": TRAINER_MODULE_FILE,
    "tuner_module": TUNER_MODULE_FILE,
    "transform_module": TRANSFORM_MODULE_FILE,
    "train_steps": 1000,
    "eval_steps": 800,
    "serving_model_dir": serving_model_dir,
})

pipeline = init_local_pipeline(components, pipeline_root)
BeamDagRunner().run(pipeline=pipeline)

INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'num_hidden_layers', 'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'dense_unit', 'default': None, 'conditions': [], 'min_value': 16, 'max_value': 256, 'step': 32, 'sampling': 'linear'}}, {'class_name': 'Float', 'config': {'name': 'dropout_rate', 'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.9, 'step': 0.1, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}}], 'values': {'num_hidden_layers': 2, 'dense_unit': 48, 'dropout_rate': 0.9, 'learning_rate': 0.001}}
INFO:absl:Best Hyperparameters are written to outputs/heart-failure-pipeline/Tuner/best_hyperparameters/22/best_hyperparameters.txt.
INFO:absl:Tuner results are written to outputs/heart-failure-pipel

Trial 10 Complete [00h 00m 10s]
val_binary_accuracy: 0.838085949420929

Best val_binary_accuracy So Far: 0.8571484088897705
Total elapsed time: 00h 01m 43s
Results summary
Results in outputs/heart-failure-pipeline/Tuner/.system/executor_execution/22/.temp/22/movie_review_kt
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 00 summary
Hyperparameters:
num_hidden_layers: 2
dense_unit: 48
dropout_rate: 0.9
learning_rate: 0.001
Score: 0.8571484088897705

Trial 05 summary
Hyperparameters:
num_hidden_layers: 3
dense_unit: 16
dropout_rate: 0.2
learning_rate: 0.0001
Score: 0.8476366996765137

Trial 07 summary
Hyperparameters:
num_hidden_layers: 2
dense_unit: 144
dropout_rate: 0.8
learning_rate: 0.0001
Score: 0.8476366996765137

Trial 03 summary
Hyperparameters:
num_hidden_layers: 1
dense_unit: 112
dropout_rate: 0.7000000000000001
learning_rate: 0.01
Score: 0.8429492115974426

Trial 01 summary
Hyperparameters:
num_hidden_layers: 2
dense_unit: 112
dropout_rate:

INFO:absl:node Trainer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.trainer.component.Trainer"
    base_type: TRAIN
  }
  id: "Trainer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "heart-failure-pipeline"
      }
    }
  }
  contexts {
    type {
      name: "pipeline_run"
    }
    name {
      field_value {
        string_value: "20240528-142013.736846"
      }
    }
  }
  contexts {
    type {
      name: "node"
    }
    name {
      field_value {
        string_value: "heart-failure-pipeline.Trainer"
      }
    }
  }
}
inputs {
  inputs {
    key: "examples"
    value {
      channels {
        producer_node_query {
          id: "Transform"
        }
        context_queries {
          type {
            name: "pipeline"
          }
          name {
            field_value {
              string_value: "heart-failure-pipeline"
            }
          }
        

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Sex_xf (InputLayer)         [(None, 3)]                  0         []                            
                                                                                                  
 ChestPainType_xf (InputLay  [(None, 5)]                  0         []                            
 er)                                                                                              
                                                                                                  
 RestingECG_xf (InputLayer)  [(None, 4)]                  0         []                            
                                                                                                  
 ExerciseAngina_xf (InputLa  [(None, 3)]                  0         []                      

INFO:absl:Function `_wrapped_model` contains input name(s) Sex_xf, ChestPainType_xf, RestingECG_xf, ExerciseAngina_xf, ST_Slope_xf, Age_xf, RestingBP_xf, Cholesterol_xf, FastingBS_xf, MaxHR_xf, Oldpeak_xf, resource with unsupported characters which will be renamed to sex_xf, chestpaintype_xf, restingecg_xf, exerciseangina_xf, st_slope_xf, age_xf, restingbp_xf, cholesterol_xf, fastingbs_xf, maxhr_xf, oldpeak_xf, model_1_dense_8_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Found untraced functions such as _update_step_xla while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to outputs/heart-failure-pipeline/Trainer/model/23/Format-Serving/fingerprint.pb


1000/1000 [==============================] - 10s 9ms/step - loss: 0.6016 - binary_accuracy: 0.6520 - val_loss: 0.4208 - val_binary_accuracy: 0.8429


INFO:absl:Feature Age has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature ChestPainType has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Cholesterol has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature ExerciseAngina has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature FastingBS has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature HeartDisease has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature MaxHR has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Oldpeak has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature RestingBP has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature RestingECG has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature ST_Slope has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature Sex has a shape dim {
  size: 1
}
. Setting to DenseTens

In [ ]:
!zip -r data.zip data/
!zip -r outputs.zip outputs/
!pip freeze > requirements.txt

  adding: data/ (stored 0%)
  adding: data/heart.csv (deflated 77%)
  adding: outputs/ (stored 0%)
  adding: outputs/heart-failure-pipeline/ (stored 0%)
  adding: outputs/heart-failure-pipeline/ExampleValidator/ (stored 0%)
  adding: outputs/heart-failure-pipeline/ExampleValidator/anomalies/ (stored 0%)
  adding: outputs/heart-failure-pipeline/ExampleValidator/anomalies/6/ (stored 0%)
  adding: outputs/heart-failure-pipeline/ExampleValidator/anomalies/6/Split-eval/ (stored 0%)
  adding: outputs/heart-failure-pipeline/ExampleValidator/anomalies/6/Split-eval/SchemaDiff.pb (deflated 53%)
  adding: outputs/heart-failure-pipeline/ExampleValidator/anomalies/6/Split-train/ (stored 0%)
  adding: outputs/heart-failure-pipeline/ExampleValidator/anomalies/6/Split-train/SchemaDiff.pb (deflated 53%)
  adding: outputs/heart-failure-pipeline/ExampleValidator/.system/ (stored 0%)
  adding: outputs/heart-failure-pipeline/ExampleValidator/.system/stateful_working_dir/ (stored 0%)
  adding: outputs/heart